<a href="https://colab.research.google.com/github/chakrateja70/RAG-AGENT/blob/main/Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_openai langchain langchain_chroma docx2txt langchain_community langchain_text_splitters pypdf weaviate-client

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.4/330.4 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import langchain
print(langchain.__version__)

0.3.12


**Required API Keys**


# *Loading PDF*

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    pages = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        pages.extend(loader.load())
    return pages

folder_path = "/content/"
pages = load_documents(folder_path)
print(f"Loaded {len(pages)} documents from the folder.")


Unsupported file type: .config
Unsupported file type: .ipynb_checkpoints
Unsupported file type: sample_data
Loaded 16 documents from the folder.


In [ ]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("/content/docs/NLP.pdf")
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

In [ ]:
pages

[Document(metadata={'source': '/content/nlp.pdf', 'page': 0, 'page_label': '1'}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡ , Ethan Perez?,\nAleksandra Piktus† , Fabio Petroni† , Vladimir Karpukhin† , Naman Goyal† , Heinrich Küttler† ,\nMike Lewis† , Wen-tau Yih† , Tim Rocktäschel†‡ , Sebastian Riedel†‡ , Douwe Kiela†\n† Facebook AI Research;‡ University College London;?New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge\nin their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP tasks. However, their ability to access and precisely manipulate\nknowledge is still limited, and hence on knowledge-intensive tasks, their perfor-\nmance lags behind task-speciﬁc architectures. Additionally, providing provenance\nfor their decisions and updating their world knowledge remain open research prob-\nlems. Pre-trained models with a differen

# *splitting text from above loaded PDF*

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=3500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
splits = text_splitter.split_documents(pages)
print(f"split the document into  {len(splits)} chunks.")

split the document into  29 chunks.


In [ ]:
print(1, splits[2].page_content)

1 For knowledge-intensive generation, we experiment with MS-MARCO [1] and Jeopardy question
generation, and we ﬁnd that our models generate responses that are more factual, speciﬁc, and
diverse than a BART baseline. For FEVER [56] fact veriﬁcation, we achieve results within 4.3% of
state-of-the-art pipeline models which use strong retrieval supervision. Finally, we demonstrate that
the non-parametric memory can be replaced to update the models’ knowledge as the world changes.1
2 Methods
We explore RAG models, which use the input sequencex to retrieve text documentsz and use them
as additional context when generating the target sequencey. As shown in Figure 1, our models
leverage two components: (i) a retrieverp⌘(z|x) with parameters⌘ that returns (top-K truncated)
distributions over text passages given a queryx and (ii) a generatorp✓(yi|x, z, y1:i 1) parametrized
1Code to run experiments with RAG has been open-sourced as part of the HuggingFace Transform-
ers Library [66] and can be fo

# *Embedding the Chunk of documents*

In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])


<ipython-input-10-41b16e96e218>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dataset

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
document_embeddings

# *Storing data in chroma*

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# Define embedding model
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Define Chroma vector database
vectordb = Chroma.from_documents(
    documents=splits,  # Ensure `splits` contains the list of document chunks
    embedding=embedding_function,
    persist_directory="./chroma.db"  # Specify directory to persist the database
)

print("Vector store created and persisted to './chroma.db'")


Vector store created and persisted to './chroma.db'


# *Performing Similarity Search*

In [ ]:

query = "what is oops?"
search_results = vectordb.similarity_search(query, k=5)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()



Top 2 most relevant chunks for the query: 'what is oops?'

Result 1:
Source: /content/nlp.pdf
Content: TheDiYineComed\(x) T
QXeU\EQcRdeU
T([)
MIPS pθ
GeneUaWoU¬Sѡ(PaUaPeWULc)
MaUgin-ali]eThiV14WhcenWXU\ZoUkiVdiYidedinWo3VecWionV:"InfeUno","PXUgaWoUio"&"PaUadiVo"(y)
End-to-End Backprop through T and¬pθ
BaUackObamaZaVboUninHaZaii.(x)FacW VeUiÀcaWiRQ: FacW QXeU\
VXppoUWV(y)
QXeVWiRQ GeQeUaWiRQ
FacW VeUiÀcaWiRQ:LabeO GeQeUaWiRQ
DRcXmeQWIQde[
Define"middleeaU"(x)
QXeVWiRQ AQVZeUiQg:QXeVWiRQ QXeU\
ThemiddleeaUinclXdeVWheW\mpaniccaYiW\andWheWhUeeoVVicleV.(y)QXeVWiRQ AQVZeUiQg:AQVZeU GeQeUaWiRQReWUieYeU Sη(NRQ-PaUaPeWULc) z4z3z2z1
d(])
JeRSaUd\ QXeVWiRQGeQeUaWiRQ:AQVZeU QXeU\
Figure 1: Overview of our approach. We combine a pre-trained retriever (Query Encoder+ Document
Index) with a pre-trained seq2seq model (Generator) and ﬁne-tune end-to-end. For queryx, we use
Maximum Inner Product Search (MIPS) to ﬁnd the top-K documentszi. For ﬁnal predictiony, we
tre

In [ ]:
2

*Retrival of relavant data*

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
retriever_results = retriever.invoke("what is nxtwave?")
print(retriever_results)


[Document(id='da244511-7674-489a-83ba-c94083d25a95', metadata={'page': 1, 'page_label': '2', 'source': '/content/nlp.pdf'}, page_content='The\x03DiYineComed\\\x03(x) T\nQXeU\\EQcRdeU\nT([)\nMIPS pθ\nGeneUaWoU¬Sѡ(PaUaPeWULc)\nMaUgin-ali]eThiV\x0314Wh\x03cenWXU\\\x03ZoUkiV\x03diYided\x03inWo\x033VecWionV:\x03"InfeUno","PXUgaWoUio"\x03&"PaUadiVo"\x03\x03\x03\x03\x03\x03\x03\x03\x03(y)\nEnd-to-End Backprop through T and¬pθ\nBaUack\x03Obama\x03ZaVboUn\x03in\x03HaZaii.(x)FacW VeUiÀcaWiRQ: FacW QXeU\\\nVXppoUWV\x03(y)\nQXeVWiRQ GeQeUaWiRQ\nFacW VeUiÀcaWiRQ:LabeO GeQeUaWiRQ\nDRcXmeQWIQde[\nDefine\x03"middle\x03eaU"(x)\nQXeVWiRQ AQVZeUiQg:QXeVWiRQ QXeU\\\nThe\x03middle\x03eaU\x03inclXdeVWhe\x03W\\mpanic\x03caYiW\\\x03andWhe\x03WhUee\x03oVVicleV.\x03\x03(y)QXeVWiRQ AQVZeUiQg:AQVZeU GeQeUaWiRQReWUieYeU Sη(NRQ-PaUaPeWULc) z4z3z2z1\nd(])\nJeRSaUd\\ QXeVWiRQGeQeUaWiRQ:AQVZeU QXeU\\\nFigure 1: Overview of our approach. We combine a pre-trained retriever (Query Encoder+ Document\nIndex) with a pre-tra